# Last x-years max finder

The idea is really similar to the "ATH finder", but instead of looking for ATH the script find if the x-year max has been beaten. Let's focus on WIG20, mWIG40 and sWIG80 here.

In [1]:
import pandas as pd
from yahoofinancials import YahooFinancials
import numpy as np
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

Read the tickers from file (ATH_Data/WIG_tickers.txt).

In [2]:
with open("ATH_Data/WIG_tickers.txt", "r") as file:
    three_wigs_tickers = file.read().splitlines()

Get data.

In [3]:
def get_data(start_date, end_date, tickers, interval):
    yahoo_financials = YahooFinancials(tickers)
    stats = yahoo_financials.get_historical_price_data(start_date, end_date, time_interval=interval)
    return stats

The starting date needs to be set as yesterday - x years and the ending date as yesterday.

In [4]:
def get_start_end_date(years):
    end = datetime.today() - timedelta(days=1)
    start = end - relativedelta(years=years)
    return start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')

In [5]:
x_years = 10
start_date, end_date = get_start_end_date(x_years)
interval = 'daily'
data = get_data(start_date, end_date, three_wigs_tickers, interval)

Create the combined data with close prices.

In [6]:
def get_close_df(dist_data):
    df_f = pd.DataFrame()
    for ticker, hist_data in dist_data.items():
        if 'prices' in hist_data.keys(): 
            df = pd.DataFrame(hist_data['prices'], columns=['close', 'formatted_date'])
            df.set_index('formatted_date', inplace=True)
            df.index = pd.to_datetime(df.index)
            df.rename(columns={'close': f'{ticker}'}, inplace=True)
            df_f = pd.concat([df_f, df], axis=1)
    return df_f

In [7]:
close_df = get_close_df(data)

In [8]:
close_df

,ALE.WA,ACP.WA,CCC.WA,CDR.WA,CPS.WA,DNP.WA,JSW.WA,KGH.WA,LTS.WA,LPP.WA,...,TOA.WA,TRK.WA,UNT.WA,VRC.WA,VGO.WA,VOX.WA,VRG.WA,WWL.WA,WLT.WA,ZEP.WA
formatted_date,,,,,,,,,,,,,,,,,,,,,
2012-05-03,NaN,46.730000,59.700001,5.390000,13.900000,NaN,92.000000,138.000000,25.797001,2820.0,...,2.88,8.72,NaN,NaN,NaN,13.750000,1.039531,563.0,3.14,NaN
2012-05-04,NaN,46.500000,63.000000,5.060000,13.500000,NaN,91.250000,135.199997,25.581575,2900.0,...,2.90,8.56,NaN,NaN,NaN,13.700000,0.990030,563.0,3.15,NaN
2012-05-07,NaN,47.950001,61.000000,5.100000,14.040000,NaN,93.400002,134.000000,25.491817,2850.5,...,2.86,8.48,NaN,NaN,NaN,13.550000,0.970229,546.5,3.04,NaN
2012-05-08,NaN,46.950001,60.599998,5.250000,13.840000,NaN,92.000000,130.100006,24.459578,2860.0,...,2.75,8.40,NaN,NaN,NaN,13.250000,0.980130,550.0,3.04,NaN
2012-05-09,NaN,47.209999,59.250000,5.200000,13.540000,NaN,90.150002,128.399994,23.382458,2875.0,...,2.78,9.04,NaN,NaN,NaN,13.000000,0.990030,550.0,3.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-26,24.615000,75.000000,50.700001,127.959999,25.799999,314.799988,65.220001,138.600006,67.580002,9920.0,...,6.01,1.71,46.500000,45.299999,684.0,40.099998,3.740000,462.0,7.10,16.000000
2022-04-27,24.049999,75.000000,50.599998,129.119995,25.260000,313.899994,66.720001,143.500000,67.400002,9700.0,...,6.00,1.69,46.599998,42.000000,678.0,39.299999,3.710000,461.0,7.02,16.240000
2022-04-28,24.350000,76.199997,53.980000,123.480003,25.559999,302.600006,69.900002,142.850006,67.900002,9830.0,...,5.91,1.69,47.000000,42.400002,666.0,39.700001,3.780000,458.0,7.00,16.500000


Create the x-years list based on historical data.

In [9]:
max_ser = close_df.max()

In [10]:
max_ser

ALE.WA      94.639999
ACP.WA      99.199997
CCC.WA     309.000000
CDR.WA     460.799988
CPS.WA      37.860001
             ...     
VOX.WA      55.599998
VRG.WA       5.280000
WWL.WA    1459.000000
WLT.WA      18.320000
ZEP.WA      33.500000
Length: 140, dtype: float64

Get data from today. Make sure the market is already closed.

In [11]:
def get_current_prices_dict(tickers):
    yahoo_financials = YahooFinancials(tickers)
    return yahoo_financials.get_current_price()

In [12]:
today_price_dict = get_current_prices_dict(three_wigs_tickers)

In [13]:
len(today_price_dict)

140

Compare with x-year max table and show new x-year max (if any).

In [14]:
def get_new_maxs(cur_max_ser, new_prices_dict):
    mask = cur_max_ser.loc[new_prices_dict.keys()] < list(new_prices_dict.values())
    return list(cur_max_ser.loc[new_prices_dict.keys()][mask].index)

In [15]:
new_max = get_new_maxs(max_ser, today_price_dict)
f'There is new {x_years} years max for: {new_max}'

"There is new 10 years max for: ['ATC.WA']"

Let's create the log to keep the track of new maxs. It is enough to have it in txt file with following scheme --> Date, Ticker, Years, Old_max, New_max.

In [16]:
def round_2(x):
    return round(x, 2)

In [17]:
def update_log_file(max_ser, new_prices_dict, new_max_tickers):
    today = datetime.today().strftime('%Y-%m-%d')
    with open('ATH_Data/x_years_max.txt', 'a') as file:
        for ticker in new_max_tickers:
            file.write(f'{today}, {ticker}, {x_years}, {round_2(max_ser[ticker])}, {round_2(new_prices_dict[ticker])}\n')

In [18]:
update_log_file(max_ser, today_price_dict, new_max)

All in one (the functions have to be already defined).

In [ ]:
x_years = 10
start_date, end_date = get_start_end_date(x_years)
interval = 'daily'
data = get_data(start_date, end_date, three_wigs_tickers, interval)
close_df = get_close_df(data)
max_ser = close_df.max()
today_price_dict = get_current_prices_dict(three_wigs_tickers)
new_max = get_new_maxs(max_ser, today_price_dict)
today = datetime.today().strftime('%Y-%m-%d')
update_log_file(max_ser, today_price_dict, new_max)
f'There is new {x_years} years max for: {new_max}'